In [0]:
df = spark.read.option('header', True).csv("dbfs:/mnt/databricks/Input")

display(df)

empid,empname,gender,country
1001,maheer1,male1,india1
1002,ati,female,usa
1003,wafa,male,india
101,Ram,Male,null
102,Sita,Female,null
103,Lakshman,Male,null
1,Vinod,null,null
2,Hari,null,null
3,Krishna,null,null
4,Ram,null,null


In [0]:
import re

# Define your Azure Blob Storage account details and paths
storage_account_name = "azdfdemostorage"
container_name = "databricks"
input_path = "dbfs:/mnt/databricks/Input"
checkpoint_path = "dbfs:/mnt/databricks/Input1"  # Adjust as needed
output_path = "dbfs:/mnt/databricks/Output1"  # Adjust as needed

# Function to clean column names
def clean_column_names(df):
    for col in df.columns:
        new_col = re.sub(r'[ ,;{}()\n\t=]', '_', col)
        df = df.withColumnRenamed(col, new_col)
    return df

# Configure Spark to read files using Auto Loader
df = (spark.readStream.format("cloudFiles")
      .option("cloudFiles.format", "csv")  # Specify the format of your files
      .option("cloudFiles.schemaLocation", "dbfs:/mnt/databricks/Output/schemalocation")
      .load(input_path))

# Clean up column names
df_cleaned = clean_column_names(df)

# Define options for writing output (adjust as needed)
query = (df_cleaned.writeStream
         .format("delta")  # Example output format (Delta Lake)
         .option("checkpointLocation", checkpoint_path)
         .option("path", output_path)
         .option("mergeSchema", "true")  # Enable schema evolution with mergeSchema option
         .start())


In [0]:
spark.conf.set("fs.azure.account.auth.type.azdfdemostorage.blob.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.azdfdemostorage.blob.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.azdfdemostorage.blob.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-07-13T00:48:33Z&st=2024-07-11T16:48:33Z&spr=https&sig=PP08YRRHhnNz3evyDPSNhsEiiM3bmJnwJNGRQcLvW4U%3D")

In [0]:
df = spark.read.csv("wasbs://databricks@azdfdemostorage.blob.core.windows.net/databricks/Input/employee1.csv", header = True)

display(df)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3812076969686821> in <cell line: 1>()
----> 1 df = spark.read.csv("wasbs://databricks@azdfdemostorage.blob.core.windows.net/databricks/Input/employee1.csv", header = True)
      2 
      3 display(df)

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumn

In [0]:
dbutils.fs.mount(
  source = "wasbs://databricks@azdfdemostorage.blob.core.windows.net",
  mount_point = "/mnt/databricks",
  extra_configs = {'fs.azure.account.key.azdfdemostorage.blob.core.windows.net':'dCKerLHku1iFJwpfqSWuajDpzIXAbLqxvfBvM0yTDnwP11imU6oenQibwaG/ahTkFvFdy89YT8dE+AStcMTD0A=='}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-3812076969686824> in <cell line: 1>()
----> 1 dbutils.fs.mount(
      2   source = "wasbs://databricks@azdfdemostorage.blob.core.windows.net",
      3   mount_point = "/mnt/databricks",
      4   extra_configs = {'fs.azure.account.key.azdfdemostorage.blob.core.windows.net':'dCKerLHku1iFJwpfqSWuajDpzIXAbLqxvfBvM0yTDnwP11imU6oenQibwaG/ahTkFvFdy89YT8dE+AStcMTD0A=='}
      5 )

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    360                     exc.__context__ = None
    361                     exc.__cause__ = None
--> 362                     raise exc
    363 
    364             return f_with_exception_handling

ExecutionError: An error occurred while calling o394.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted

In [0]:
df = spark.read.csv("/mnt/databricks/Input/employee1.csv",header=True)
display(df)

empid,empname,gender,country
1001,maheer1,male1,india1
1002,ati,female,usa
1003,wafa,male,india


In [0]:
input_stream = "dbfs:/mnt/databricks/Input"

In [0]:
output_stream = "dbfs:/mnt/databricks/Output"

In [0]:
%sql
use default

In [0]:
(spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.inferColumnTypes", True)
    .option("cloudFiles.schemaEvolutionMode","rescue")
    .option("cloudFiles.schemaLocation", f"{output_stream}/ram/autoloader/schemalocation")
    .load(f"{input_stream}")
    .writeStream
    .option("checkpointLocation", f"{output_stream}/ram/autoloader/checkpoint")
    .option("path", f"{output_stream}/ram/autoloader/output")
    .option('mergeSchema', True)
    .table("autoloader")
)


Out[7]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f8f14a708e0>

In [0]:
%sql
select * from default.autoloader

empid,empname,gender,country,_rescued_data
1001,maheer1,male1,india1,null
1002,ati,female,usa,null
1003,wafa,male,india,null


In [0]:
%sql
describe extended default.autoloader

col_name,data_type,comment
empid,string,null
empname,string,null
gender,string,null
country,string,null
_rescued_data,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,autoloader,


In [0]:
%sql
create table default.my_table

In [0]:
%sql
COPY INTO my_table
FROM 'dbfs:/mnt/databricks/Input/'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true')

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
10,10,0


In [0]:
%sql
select * from my_table

empid,empname,gender,country
1001,maheer1,male1,india1
1002,ati,female,usa
1003,wafa,male,india
101,Ram,Male,null
102,Sita,Female,null
103,Lakshman,Male,null
1,Vinod,null,null
2,Hari,null,null
3,Krishna,null,null
4,Ram,null,null
